Дорогой студент!

В домашнем задании Lite вам предлагается поработать подробнее с параметрами словаря и формированием гиперпараметров нейронной сети. Создайте 9 нейросетей с различными гиперпараметрами (см. пунтк 2 и 3)

 Для этого необходимо:

  1. Воссоздать ноутбук, аналогичный ноутбуку практической части №1, загрузив при этом необходимую нам базу (код уже доступен в ноутбуке).

  2. Задать в ноутбуке следующие параметры для размера словаря, ширины окна и шага:

    - Размер словаря - от 10000 до 20000 (выбрать меньшее значение диапазона, если будет перегрузка ОЗУ и перезапуск подключения к Colaboratory)
    - Ширина окна - от 1000 до 2000
    - Шаг - от 100 до 500 (на обучение лучше влияет наименьший шаг, но это может перегрузить ОЗУ).

  3. Создать архитектуру сети и задать гиперпараметры. Можно воспользоваться шаблоном:
  
   - Добавьте модель прямого распространения **Sequential()**
   - Добавьте один или несколько полносвязных (**Dense**) слоёв
   - Добавьте слои **Dropout()** и **BatchNormalization()**
   - Добавьте выходной полносвязный слой с количеством нейронов, соответствующим количеству классов (число писателей)
  
   Напомним, что точность сети можно проверить по значению показателя 'val_accuracy' на конце каждой эпохи.
   

### Импорт библиотек

In [ ]:
# Работа с массивами данных
import numpy as np
import tensorflow as tf
# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils
# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential
# Основные слои
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer
# Загрузка датасетов из облака google
import gdown
import pandas as pd
# Функции операционной системы
import os
# Регулярные выражения
import re

In [ ]:
# Работа с google диском
from google.colab import drive

### Установка сида

In [ ]:
SEED = 52
os.environ['PYTHONHASHSEED']=str(SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
np.random.seed(SEED)
tf.random.set_seed(SEED)

Загрузка датасета с текстами писателей в виде архива и распаковка его в папку:

In [ ]:
# Загрузка датасета из облака
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l7/writers.zip', None, quiet=True)

# Распаковка архива в папку writers
!unzip -qo writers.zip -d writers/

# Просмотр содержимого папки
!ls writers

'(Булгаков) Обучающая_5 вместе.txt'	    '(О. Генри) Обучающая_50 вместе.txt'
'(Булгаков) Тестовая_2 вместе.txt'	    '(О. Генри) Тестовая_20 вместе.txt'
'(Клиффорд_Саймак) Обучающая_5 вместе.txt'  '(Рэй Брэдберри) Обучающая_22 вместе.txt'
'(Клиффорд_Саймак) Тестовая_2 вместе.txt'   '(Рэй Брэдберри) Тестовая_8 вместе.txt'
'(Макс Фрай) Обучающая_5 вместе.txt'	    '(Стругацкие) Обучающая_5 вместе.txt'
'(Макс Фрай) Тестовая_2 вместе.txt'	    '(Стругацкие) Тестовая_2 вместе.txt'


Сохранение в отдельных переменных:

   - Имя для папки с текстами
   - Название выборки "обучающая"
   - Название выборки "тестовая"

In [ ]:
# Папка с текстовыми файлами
FILE_DIR  = 'writers'
# Признак обучающей выборки в имени файла
SIG_TRAIN = 'обучающая'
# Признак тестовой выборки в имени файла
SIG_TEST  = 'тестовая'

**Шаг 2.** Преобразование базы данных для обучения.

Добавление имен всех писателей в список классов.

Преобразование текстов в одну длинную строку и разбиение их отдельно в списки для каждого класса и выборки.

In [ ]:
# В список добавляются имена классов
CLASS_LIST = []

# Здесь сохраняются тексты для обучения сети
text_train = []

# А здесь для проверки точности сети
text_test = []

# цикл для итерации по каждому имени текста в общей папке
for file_name in os.listdir(FILE_DIR):

    # Выделение имени класса и типа выборки из имени файла
    m = re.match('\((.+)\) (\S+)_', file_name)

    # Если выделение получилось, то файл обрабатывается
    if m:
        # отдельно берём имя класса (автора)
        class_name = m[1]
        # отдельно - имя выборки
        subset_name = m[2].lower()
        # Проверка типа выборки по названию в имени файла
        is_train = SIG_TRAIN in subset_name
        is_test = SIG_TEST in subset_name
        # Если тип выборки обучающая или тестовая - файл обрабатывается
        if is_train or is_test:
            # Добавление нового класса, если его еще нет в списке
            if class_name not in CLASS_LIST:
                # Выводится информационное сообщение о добавлении названия класса
                CLASS_LIST.append(class_name)
                # Инициализация соответствующих классу строк текста
                text_train.append('')
                text_test.append('')

            # Поиск индекса класса для добавления содержимого файла в выборку
            cls = CLASS_LIST.index(class_name)
            # Выводится информационное сообщение о добавлении класса в список классов и текста к выборке
            print(f'Добавление файла "{file_name}" в класс "{CLASS_LIST[cls]}", {subset_name} выборка.')
            # оператор with - безопасное чтение каждого файла с текстом
            with open(f'{FILE_DIR}/{file_name}', 'r') as f:
                # Загрузка содержимого файла в строку
                text = f.read()
            # Определение выборки, куда будет добавлено содержимое
            subset = text_train if is_train else text_test
            # Добавление текста к соответствующей выборке класса. Концы строк заменяются на пробел
            subset[cls] += ' ' + text.replace('\n', ' ')

# Кол-во классов
CLASS_COUNT = len(CLASS_LIST)

Добавление файла "(Стругацкие) Тестовая_2 вместе.txt" в класс "Стругацкие", тестовая выборка.
Добавление файла "(Клиффорд_Саймак) Обучающая_5 вместе.txt" в класс "Клиффорд_Саймак", обучающая выборка.
Добавление файла "(Рэй Брэдберри) Обучающая_22 вместе.txt" в класс "Рэй Брэдберри", обучающая выборка.
Добавление файла "(Рэй Брэдберри) Тестовая_8 вместе.txt" в класс "Рэй Брэдберри", тестовая выборка.
Добавление файла "(Макс Фрай) Тестовая_2 вместе.txt" в класс "Макс Фрай", тестовая выборка.
Добавление файла "(Булгаков) Тестовая_2 вместе.txt" в класс "Булгаков", тестовая выборка.
Добавление файла "(Стругацкие) Обучающая_5 вместе.txt" в класс "Стругацкие", обучающая выборка.
Добавление файла "(Макс Фрай) Обучающая_5 вместе.txt" в класс "Макс Фрай", обучающая выборка.
Добавление файла "(О. Генри) Тестовая_20 вместе.txt" в класс "О. Генри", тестовая выборка.
Добавление файла "(О. Генри) Обучающая_50 вместе.txt" в класс "О. Генри", обучающая выборка.
Добавление файла "(Булгаков) Обучающая_5 

Подготовка текстовой инфрмации для обработки Токенайзером. Для этого сохрание в подходящих именах переменных со следующим параметрами:

   - Объем словаря для токенизатора
   - Длина отрезка текста (окна) в словах
   - Шаг смещения окна по тексту для деления на векторы

И затем вызов самого Токенайзера, передав в его параметры эти переменные и оставшиеся значения для преобразования.

In [ ]:
# Объем словаря для токенизатора
VOCAB_SIZE = 15000
# Длина отрезка текста (окна) в словах
WIN_SIZE   = 3000
# Шаг окна разбиения текста на векторы
WIN_HOP    = 300

# Токенайзер из Keras для разбиения текста и построения частотного словаря
tokenizer = Tokenizer(num_words=VOCAB_SIZE, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
                          lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)

Отдельно обучение самого Токенайзера на выборке текстов для обучения сети:


In [ ]:
# Получаем словарь частотности
tokenizer.fit_on_texts(text_train)

Преобразование обучающих и проверочных текстов в последовательность индексов согласно частотному словарю Токенайзера:

In [ ]:
seq_train = tokenizer.texts_to_sequences(text_train)
seq_test = tokenizer.texts_to_sequences(text_test)

**Шаг 3.** Создание функций для формирования выборок. Формирование выборок.

Воспроизведение блока кода с двумя функциями:

  1. Функция деления последовательности индексов на отрезки скользящим окном

In [ ]:
def split_sequence(sequence,   # Последовательность индексов
                   win_size,   # Размер окна для деления на примеры
                   hop):       # Шаг окна

    # Последовательность разбивается на части до последнего полного окна
    return [sequence[i:i + win_size] for i in range(0, len(sequence) - win_size + 1, hop)]

 2. Функция формирования выборок из индексов и соответствующих классам меток в формате One Hot Encoding

In [ ]:
def vectorize_sequence(seq_list,    # Список последовательностей индексов
                       win_size,    # Размер окна для деления на примеры
                       hop):        # Шаг окна

    # В списке последовательности следуют в порядке их классов (их кол-во сповпадает с кол-вом классов)
    class_count = len(seq_list)
    # Списки для исходных векторов и категориальных меток класса
    x, y = [], []
    # Для каждого класса:
    for cls in range(class_count):
        # Разбиение последовательности класса cls на отрезки
        vectors = split_sequence(seq_list[cls], win_size, hop)
        # Добавление отрезков в выборку
        x += vectors
        # Для всех отрезков класса cls добавление меток класса в виде OHE
        y += [utils.to_categorical(cls, class_count)] * len(vectors)

    # Возврат результатов как numpy-массивов
    return np.array(x), np.array(y)

Формирование выборки (x_train, y_train) и (x_test, y_test) с помощью функций, воспроизведённых в ячейке выше:

In [ ]:
# Формирование обучающей выборки
x_train, y_train = vectorize_sequence(seq_train, WIN_SIZE, WIN_HOP)
# Формирование тестовой выборки
x_test, y_test = vectorize_sequence(seq_test, WIN_SIZE, WIN_HOP)

# Проверка формы сформированных данных
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(5842, 3000) (5842, 6)
(2190, 3000) (2190, 6)


Формирование выборки индексов слов в виде матриц Bag Of Words методом  **sequences_to_matrix( )**.

In [ ]:
# На входе .sequences_to_matrix() ожидает список, .tolist() выполняет преобразование к типу данных 'список'
x_train_01 = tokenizer.sequences_to_matrix(x_train.tolist())
x_test_01 = tokenizer.sequences_to_matrix(x_test.tolist())

# Выводим форму обучающей выборки в виде матрицы Bag of Words
print(x_train_01.shape)
# Выводим фрагмент отрезка обучающего текста в виде Bag of Words
print(x_train_01[0][0:100])

(5842, 15000)
[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]


**Шаг 4.** Создание архитектуры нейронной сети.

Создайте несложную архитектуру нейронной сети. После этого произведите компиляцию модели с соответствующими параметрами:

In [ ]:
model = Sequential()                                            # Создание последовательной модели нейросети
model.add(Dense(184, input_dim=VOCAB_SIZE, activation="relu"))  # Первый полносвязный слой
model.add(Dense(86, activation="relu"))                         # Второй полносвязный слой
model.add(Dense(33, activation="relu"))                         # Третий полносвязный слой
model.add(BatchNormalization())                                 # Слой пакетной нормализации
model.add(Dropout(0.1))                                         # Слой регуляризации Dropout
model.add(Dense(CLASS_COUNT, activation='sigmoid'))             # Выходной полносвязный слой

model.compile(optimizer='adam',                                 # Компиляция модели для обучения на данных вида Bag of Words
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Обучите модель нейронной сети и посмотрите на результаты точности на проверочной выборке:

In [ ]:
# Обучение сети с помощью функции fit()
history = model.fit(x_train_01,                            # Обучающая выборка Bag of Words
                        y_train,                               # Метки классов обучающей выборки
                        epochs=20,                             # Количество эпох
                        batch_size=32,                         # Размер подвыборки для одного шага по данным на эпохе
                        validation_data=(x_test_01, y_test))   # Проверочная выборка и метки классов проверочной выборки

Epoch 1/20
183/183 [==============================] - 7s 10ms/step - loss: 0.1003 - accuracy: 0.9832 - val_loss: 0.3200 - val_accuracy: 0.9370
Epoch 2/20
183/183 [==============================] - 1s 6ms/step - loss: 0.0069 - accuracy: 1.0000 - val_loss: 0.3051 - val_accuracy: 0.9178
Epoch 3/20
183/183 [==============================] - 1s 7ms/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.2929 - val_accuracy: 0.9137
Epoch 4/20
183/183 [==============================] - 1s 7ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.2797 - val_accuracy: 0.9146
Epoch 5/20
183/183 [==============================] - 1s 8ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.2700 - val_accuracy: 0.9128
Epoch 6/20
183/183 [==============================] - 2s 8ms/step - loss: 9.0407e-04 - accuracy: 1.0000 - val_loss: 0.2385 - val_accuracy: 0.9320
Epoch 7/20
183/183 [==============================] - 1s 6ms/step - loss: 6.8182e-04 - accuracy: 1.0000 - val_loss: 0.2384 - val_accuracy: 0.92

Создать 9 нейросетей с различными параметрами словаря и гиперпараметрами нейронной сети.

Параметры размера словаря, ширины окна и шага:
- Размер словаря - от 10000 до 20000 (выбрать меньшее
значение диапазона, если будет перегрузка ОЗУ и перезапуск подключения к Colaboratory)
- Ширина окна - от 1000 до 2000
- Шаг - от 100 до 500 (меньший шаг лучше для обучения, но это может перегрузить ОЗУ).




Архитектура сети с гиперпараметрами:
- Модель прямого распространения Sequential()
- Один или несколько полносвязных (Dense) слоёв
- Слои Dropout() и BatchNormalization()
- Выходной полносвязный слой с количеством нейронов, соответствующим количеству классов (число писателей)
Точность сети проверить по значению показателя 'val_accuracy'.

Запишите результаты в таблицу:

In [ ]:
CLASS_COUNT = len(CLASS_LIST)
VOCAB_SIZE = [10000, 15000]
WIN_SIZE = [1000, 2000]
WIN_HOP = [200, 500]
table = pd.DataFrame()

def split_sequence(sequence, win_size, hop):
    return [sequence[i:i + win_size] for i in range(0, len(sequence) - win_size + 1, hop)]

def vectorize_sequence(seq_list, win_size, hop):
    class_count = len(seq_list)
    x, y = [], []

    for cls in range(class_count):
        vectors = split_sequence(seq_list[cls], win_size, hop)
        x += vectors
        y += [utils.to_categorical(cls, class_count)] * len(vectors)

    return np.array(x), np.array(y)


for i in VOCAB_SIZE:
  for j in WIN_SIZE:
    for k in WIN_HOP:
      for l in [True, False]:
        tokenizer = Tokenizer(num_words=i, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
                            lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)

        tokenizer.fit_on_texts(text_train)

        seq_train = tokenizer.texts_to_sequences(text_train)
        seq_test = tokenizer.texts_to_sequences(text_test)

        x_train, y_train = vectorize_sequence(seq_train, j, k)
        x_test, y_test = vectorize_sequence(seq_test, j, k)

        x_train_01 = tokenizer.sequences_to_matrix(x_train.tolist())
        x_test_01 = tokenizer.sequences_to_matrix(x_test.tolist())

        model = Sequential()
        model.add(Dense(256, input_dim=i, activation="relu"))
        if l:
          model.add(Dense(128, activation="relu"))
        model.add(Dense(32, activation="relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.1))
        model.add(Dense(CLASS_COUNT, activation='sigmoid'))

        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        history = model.fit(x_train_01, y_train, epochs=10, batch_size=64,
                                validation_data=(x_test_01, y_test), verbose=0)


        valAccMaxInd = np.argmax(history.history['val_accuracy'])
        # print(history.history)
        lossMax = history.history['loss'][valAccMaxInd]
        accMax = history.history['accuracy'][valAccMaxInd]
        valLossMax = history.history['val_loss'][valAccMaxInd]
        valAccMax = history.history['val_accuracy'][valAccMaxInd]

        print(np.argmax(history.history['val_accuracy']))


        historySrs = pd.Series([lossMax,accMax,valLossMax,valAccMax],
                              index=['loss','accuracy','val_loss','val_accuracy'])

        print(f'{historySrs}')
        table[f'VOCAB_SIZE:{i} WIN_SIZE:{j} WIN_HOP:{k} Sec_Dence:{l}'] = historySrs

table
print(f'Вывод: Лучший результат {round(table[table.idxmax(axis="columns")[3]]["val_accuracy"], 2)}% при параметрах {table.idxmax(axis="columns")[3]}')

9
loss            0.000608
accuracy        1.000000
val_loss        0.345793
val_accuracy    0.891148
dtype: float64
4
loss            0.001331
accuracy        1.000000
val_loss        0.358687
val_accuracy    0.886663
dtype: float64
7
loss            0.003493
accuracy        1.000000
val_loss        0.413964
val_accuracy    0.881343
dtype: float64
7
loss            0.003533
accuracy        1.000000
val_loss        0.360254
val_accuracy    0.897015
dtype: float64
7
loss            0.000755
accuracy        1.000000
val_loss        0.269882
val_accuracy    0.926071
dtype: float64
7
loss            0.000597
accuracy        1.000000
val_loss        0.271912
val_accuracy    0.930597
dtype: float64
3
loss            0.005794
accuracy        1.000000
val_loss        0.282623
val_accuracy    0.941265
dtype: float64
6
loss            0.002699
accuracy        1.000000
val_loss        0.299893
val_accuracy    0.922440
dtype: float64
2
loss            0.003971
accuracy        1.000000
val_loss    

In [ ]:
table

,VOCAB_SIZE:10000 WIN_SIZE:1000 WIN_HOP:200 Sec_Dence:True,VOCAB_SIZE:10000 WIN_SIZE:1000 WIN_HOP:200 Sec_Dence:False,VOCAB_SIZE:10000 WIN_SIZE:1000 WIN_HOP:500 Sec_Dence:True,VOCAB_SIZE:10000 WIN_SIZE:1000 WIN_HOP:500 Sec_Dence:False,VOCAB_SIZE:10000 WIN_SIZE:2000 WIN_HOP:200 Sec_Dence:True,VOCAB_SIZE:10000 WIN_SIZE:2000 WIN_HOP:200 Sec_Dence:False,VOCAB_SIZE:10000 WIN_SIZE:2000 WIN_HOP:500 Sec_Dence:True,VOCAB_SIZE:10000 WIN_SIZE:2000 WIN_HOP:500 Sec_Dence:False,VOCAB_SIZE:15000 WIN_SIZE:1000 WIN_HOP:200 Sec_Dence:True,VOCAB_SIZE:15000 WIN_SIZE:1000 WIN_HOP:200 Sec_Dence:False,VOCAB_SIZE:15000 WIN_SIZE:1000 WIN_HOP:500 Sec_Dence:True,VOCAB_SIZE:15000 WIN_SIZE:1000 WIN_HOP:500 Sec_Dence:False,VOCAB_SIZE:15000 WIN_SIZE:2000 WIN_HOP:200 Sec_Dence:True,VOCAB_SIZE:15000 WIN_SIZE:2000 WIN_HOP:200 Sec_Dence:False,VOCAB_SIZE:15000 WIN_SIZE:2000 WIN_HOP:500 Sec_Dence:True,VOCAB_SIZE:15000 WIN_SIZE:2000 WIN_HOP:500 Sec_Dence:False
loss,0.000608,0.001331,0.003493,0.003533,0.000755,0.000597,0.005794,0.002699,0.003971,0.000722,0.002423,0.002243,0.135027,0.000298,0.002739,0.003418
accuracy,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.979527,1.000000,1.000000,1.000000
val_loss,0.345793,0.358687,0.413964,0.360254,0.269882,0.271912,0.282623,0.299893,0.363712,0.347029,0.412551,0.348753,0.411632,0.240636,0.305666,0.356741
val_accuracy,0.891148,0.886663,0.881343,0.897015,0.926071,0.930597,0.941265,0.922440,0.895634,0.899222,0.885075,0.912687,0.929089,0.937538,0.928464,0.929217
